***GENERATED CODE FOR stdscalartestts PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StandardScaler
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, udf, unix_timestamp
import pyspark
from pyspark.sql.functions import col, round
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import DoubleType, IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ConvertToDate(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]
    dateFormate = transformationData["dateFormat"]
    dfReturn = dfReturn.withColumn(feature, unix_timestamp(feature, dateFormate)
                                   .cast("double").cast("timestamp"))
    dfReturn = dfReturn.withColumn(
        feature + '_dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature + '_month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature + '_year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def vectorAssemblerTransform(df, param):

    dfReturn = df

    if (type(param) == str):
        outcol = param + "_vector"
        assembler = VectorAssembler(inputCols=[param], outputCol=outcol)
        dfReturn = assembler.transform(dfReturn)
        return dfReturn

    if (type(param) == list):
        vecAssembler = VectorAssembler(inputCols=param, outputCol="features")
        new_df = vecAssembler.transform(df)
        return new_df


def to_array(col):
    def to_array_(v):
        return v.toArray().tolist()
    return udf(to_array_, ArrayType(DoubleType()))(col)


def standardScalarTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: '0.0'})
    scalarFlags = transformationData["std_scalar"]
    if scalarFlags["mean_flag"]:
        stdflag = False
        meanflag = True
    elif scalarFlags["std_flag"]:
        stdflag = True
        meanflag = False
    outcol = feature + "_standardscalar"
    featureVector = feature + "_vector"
    dfReturn = vectorAssemblerTransform(dfReturn, feature)

    standardscale = StandardScaler(inputCol=featureVector, outputCol=outcol, withStd=stdflag,
                                   withMean=meanflag)
    scaledata = standardscale.fit(dfReturn).transform(dfReturn)
    dfReturn = scaledata.withColumn("final_col", to_array(scaledata[outcol]))\
        .select(scaledata.schema.names + [col("final_col")[0]])

    dfReturn = dfReturn.drop(outcol).drop(featureVector)\
        .withColumnRenamed("final_col[0]", outcol)
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[outcol], 2))
    return dfReturn


class TransformationTimeSeriesForecastingMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]['featureList']
        ForecastFE = configObj["FE"]
        featuresSelectedList = [ForecastFE['features']
                                ['timecolumn'], ForecastFE['features']['tocompare']]
        transformedDF = transformationDF.select(
            [c for c in transformationDF.columns if c in featuresSelectedList])
        transformedDF = CleanseData().replaceNullValues(featureData, transformedDF)
        transformationDF = ConvertToDate(transformationDF, {'transformationsData': [{'feature_label': 'AdmitDate', 'dateFormat': 'dd/MM/yyyy', 'transformation_label': 'Convert to Date'}], 'feature': 'AdmitDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                         'count': '109', 'mean': '', 'stddev': '', 'min': '01/01/2012', 'max': '31/01/2012', 'missing': '0', 'distinct': '37'}, 'transformation': [{'transformation': 'Convert to Date', 'selectedAsDefault': 1}], 'updatedLabel': 'AdmitDate'}, {'feature_label': 'AdmitDate', 'dateFormat': 'dd/MM/yyyy', 'transformation_label': 'Convert to Date'})
        transformationDF = transformationDF.drop('AdmitDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '109', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0', 'distinct': '2'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'gender'}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = standardScalarTransform(transformationDF, {'transformationsData': [{'feature_label': 'dialysisrenalendstage', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'True', 'both': 'True'}, 'transformation_label': 'Standard Scalar'}], 'feature': 'dialysisrenalendstage', 'transformation': [{'transformation': 'Standard Scalar', 'selectedAsDefault': 1}], 'type': 'numeric', 'replaceby': 'mean', 'selected': 'True', 'stats': {
            'count': '109', 'mean': '0.02', 'stddev': '0.13', 'min': '0', 'max': '1', 'missing': '0'}, 'updatedLabel': 'dialysisrenalendstage'}, {'feature_label': 'dialysisrenalendstage', 'std_scalar': {'mean_flag': 'True', 'std_flag': 'True', 'both': 'True'}, 'transformation_label': 'Standard Scalar'})
        transformationDF = transformationDF.drop('dialysisrenalendstage')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'facid', 'transformation_label': 'String Indexer'}], 'feature': 'facid', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '109', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'E', 'missing': '0', 'distinct': '5'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'facid'}, {'feature_label': 'facid', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('facid')

        transformedDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
        df = transformedDF.toPandas()
        return df


***AUTOML FUNCTIONS***

In [ ]:
from prophet import Prophet
import pandas as pd


def driverProphet(df):
    df.columns = ['ds', 'y']
    df['ds'] = df['ds'].astype(str).str[:-6]
    df['ds'] = pd.to_datetime(df['ds'])
    m = Prophet()
    m.fit(df)
    future = m.make_future_dataframe(periods=365)
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
    m.plot(forecast)
    return m


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run stdscalartesttsHooks.ipynb
try:
	#sourcePreExecutionHook()

	hlossamplecngy = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/testdata/1716978014893_12months3duplicateddmmyyyy.csv', 'filename': 'HLOSSample.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'FilePath': '/SampleSolutions/HospitalLOSTS/HLOSSample.csv', 'viewFileName': 'HLOSSample.csv', 'is_header': 'Use Header Line', 'baseType': 'hdfs', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/', 'results_url': 'http://dnm.bfirst.ai:44040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run stdscalartesttsHooks.ipynb
try:
	#transformationPreExecutionHook()

	stdscalartesttsfeatureforecast = TransformationTimeSeriesForecastingMain.run(hlossamplecngy, json.dumps( {"FE": {"functionList": [{"function": "log"}, {"function": "Original"}], "stage_attributes": {}, "featureList": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "_c0", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "4677.96", "stddev": "2820.26", "min": "0", "max": "9612", "missing": "0"}, "updatedLabel": "_c0"}, {"transformationsData": [{"feature_label": "AdmitDate", "dateFormat": "dd/MM/yyyy", "transformation_label": "Convert to Date"}], "feature": "AdmitDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "109", "mean": "", "stddev": "", "min": "01/01/2012", "max": "31/01/2012", "missing": "0", "distinct": "37"}, "transformation": [{"transformation": "Convert to Date", "selectedAsDefault": 1}], "updatedLabel": "AdmitDate"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "lengthofstay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "2.52", "stddev": "2.92", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "lengthofstay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rcount", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.65", "stddev": "1.4", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "rcount"}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "109", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "gender"}, {"transformationsData": [{"feature_label": "dialysisrenalendstage", "std_scalar": {"mean_flag": "True", "std_flag": "True", "both": "True"}, "transformation_label": "Standard Scalar"}], "feature": "dialysisrenalendstage", "transformation": [{"transformation": "Standard Scalar", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "dialysisrenalendstage"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "asthma", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "asthma"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "irondef", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.04", "stddev": "0.19", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "irondef"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pneum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "pneum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "substancedependence", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.05", "stddev": "0.21", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "substancedependence"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychologicaldisordermajor", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.12", "stddev": "0.33", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "psychologicaldisordermajo..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "depress", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.01", "stddev": "0.1", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "depress"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "psychother", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.04", "stddev": "0.19", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "psychother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fibrosisandother", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "fibrosisandother"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "malnutrition", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "malnutrition"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hemo", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "0.05", "stddev": "0.21", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "hemo"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "hematocrit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "11.22", "stddev": "1.9", "min": "7.0", "max": "23.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "hematocrit"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "neutrophils", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "9.86", "stddev": "3.47", "min": "3.0", "max": "27.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "neutrophils"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "sodium", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "137.39", "stddev": "2.77", "min": "130.0", "max": "143.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "sodium"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "glucose", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "142.08", "stddev": "29.32", "min": "75.0", "max": "217.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "glucose"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bloodureanitro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "12.37", "stddev": "6.56", "min": "4.0", "max": "64.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bloodureanitro"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "0.78", "stddev": "0.42", "min": "0.0", "max": "1.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "creatinine"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "29.24", "stddev": "1.86", "min": "24.0", "max": "33.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "bmi"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "pulse", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "74.5", "stddev": "11.8", "min": "44", "max": "111", "missing": "0"}, "updatedLabel": "pulse"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "respiration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "109", "mean": "6.01", "stddev": "0.35", "min": "4.0", "max": "7.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "respiration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "secondarydiagnosisnonicd9", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "109", "mean": "2.48", "stddev": "2.33", "min": "0", "max": "10", "missing": "0"}, "updatedLabel": "secondarydiagnosisnonicd9"}, {"transformationsData": [{"feature_label": "facid", "transformation_label": "String Indexer"}], "feature": "facid", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "109", "mean": "", "stddev": "", "min": "A", "max": "E", "missing": "0", "distinct": "5"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "facid"}], "features": {"timecolumn": "AdmitDate_converttodate", "tocompare": "lengthofstay"}, "dataPercentage": "100", "originalfile": "/FileStore/platform/testdata/1716978014893_12months3duplicateddmmyyyy.csv", "statFunction": {"function": "Original", "parameter": ""}}}))

	#transformationPostExecutionHook(stdscalartesttsfeatureforecast)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run stdscalartesttsHooks.ipynb
try:
	#mlPreExecutionHook()

	model = driverProphet(stdscalartesttsfeatureforecast)

	#mlPostExecutionHook(model)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
try:
        future = model.make_future_dataframe(periods=30,freq='MS',include_history=False)
        predictedDataFrame = model.predict(future)
        display(model.plot(predictedDataFrame))
        display(model.plot_components(predictedDataFrame))
        display(predictedDataFrame)
except Exception as ex:
    logging.error(ex)

spark.stop()


